In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import clear_output
from IPython.display import HTML

import pickle
# from google.colab import files

In [0]:
import numpy as np
from pathlib import Path
from torchvision.datasets import CIFAR10, MNIST
from torchvision import transforms
from torch.utils.data import DataLoader
import torch
import torchvision.utils as vutils
from torch import nn

In [0]:
# Setting up constants
# device
ngpu = 2

# single image
imageSize = 64
imageMean = (0.4923172 , 0.48307145, 0.4474483)
imageStd = (0.24041407, 0.23696952, 0.25565723)

# data loader
numWorkers = 1
batchSize = 8

# Network Arch
nc = 3 # Number of channels
nz = 100 # Latent vector
ngf = 128 # relates to the depth of feature maps carried through the generator
ndf = 64 # sets the depth of feature maps propagated through the discriminator

# Training
num_epochs = 5

# Adam Optimizer
lr = .0005
beta1 = .5

# convention of the labeling for the real and the fake datasets
real_label = .9
fake_label = 0

In [0]:
dataFolder = Path("./data")
# Decide which device we want to run on
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

In [0]:
# functions 
def visual_data(batch, device):
    """
    Visualize a batch of image data
    
    Params
    --------------------------------
    batch:: a batch of images to visualize
            in numpy matrix format
            
    device:: cpu/gpu device used for computations
    """
    plt.figure(figsize=(12, 12))
    plt.axis("off")
    plt.title("Training Images")
    plt.imshow(np.transpose(vutils.make_grid(batch[0].to(device)[:64], 
                                             padding=2, normalize=True).cpu(),(1,2,0)))

def load_mnist(mnistFolder, tsfms, batchSize=64, numWorkers=0):
    """
    Load, download if necessary, the CIFAR10 data and 
    return the iterable batch generator object
    
    Params
    --------------------------------
    cifarFolder:: folder to store/load the CIFAR10 data from
    tsfms:: transformations requaired to apply on data
    batchSize:: number of samples on each batch of the data
    numWorkers:: number of cpu cores used to load the data
    
    Return:
    --------------------------------
    trainLoader:: iterable training data batch generator object
    testLoader:: iterable test data batch generator object
    """
    trainData = MNIST(mnistFolder, download=True, train=True, transform=tsfms)
    trainLoader = DataLoader(trainData, batch_size=batchSize, 
                             num_workers=numWorkers, shuffle=True)
    
    # testData = MNIST(mnistFolder, download=True, train=False, transform=tsfms)   
    # testLoader = DataLoader(testData, batch_size=batchSize, num_workers=numWorkers)
    
    return trainLoader, testLoader

def load_cifar10(cifarFolder, tsfms, batchSize=64, numWorkers=0):
    """
    Load, download if necessary, the CIFAR10 data and 
    return the iterable batch generator object
    
    Params
    --------------------------------
    cifarFolder:: folder to store/load the CIFAR10 data from
    tsfms:: transformations requaired to apply on data
    batchSize:: number of samples on each batch of the data
    numWorkers:: number of cpu cores used to load the data
    
    Return:
    --------------------------------
    trainLoader:: iterable training data batch generator object
    testLoader:: iterable test data batch generator object
    """
    trainData = CIFAR10(cifarFolder, download=True, train=True, transform=tsfms)
    trainLoader = DataLoader(trainData, batch_size=batchSize, num_workers=numWorkers)
    
    testData = CIFAR10(cifarFolder, download=True, train=False, transform=tsfms)   
    testLoader = DataLoader(testData, batch_size=batchSize, num_workers=numWorkers)
    
    return trainLoader, testLoader

def weights_init(m):
    """
    Custom weight initializer from the DCGAN papar
    Different from the usual way of initializing the weigths
    
    Params:
    -----------------------------------
    m:: network object
    
    -----------------------------------
    DCGAN paper arXiv:1511.06434 
    """
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

In [0]:
# the generator

class Generator(nn.Module):
    def __init__(self, ngpu):
        super(Generator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d( nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            # state size. (ngf*8) x 4 x 4
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # state size. (ngf*4) x 8 x 8
            nn.ConvTranspose2d( ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # state size. (ngf*2) x 16 x 16
            nn.ConvTranspose2d( ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # state size. (ngf) x 32 x 32
            nn.ConvTranspose2d( ngf, nc, 4, 2, 1, bias=False),
            nn.Tanh()
            # state size. (nc) x 64 x 64
        )

    def forward(self, input):
        return self.main(input)

In [0]:
class Discriminator(nn.Module):
    def __init__(self, ngpu):
        super(Discriminator, self).__init__()
        self.ngpu = ngpu
        self.features1 = nn.Sequential(
            # input is (nc) x 64 x 64
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf) x 32 x 32
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*2) x 16 x 16
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*4) x 8 x 8
        )

        self.features2 = nn.Sequential(
            # state size. (ndf*4) x 8 x 8
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*8) x 4 x 4
        )
        
        self.classifier =  nn.Sequential(
            # state size. (ndf*8) x 4 x 4
            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
      f1 = self.features1(input)
      f2 = self.features2(f1)
      cls = self.classifier(f2)
      return cls, f1, f2

In [344]:
tsfms = transforms.Compose([
    transforms.Resize(imageSize), 

    transforms.RandomHorizontalFlip(p=0.5), 

    transforms.ToTensor(),
    transforms.Normalize(imageMean, imageStd)
])

cifarFolder = dataFolder/"CIFAR10"

trainLoader, test_loader = load_cifar10(cifarFolder, tsfms, batchSize, numWorkers)
# trainLoader, test_loader = load_mnist(cifarFolder, tsfms, batchSize, numWorkers)

Files already downloaded and verified
Files already downloaded and verified


In [0]:
gen = Generator(ngpu).to(device)
gen.apply(weights_init);
disc = Discriminator(ngpu).to(device)
disc.apply(weights_init);

In [0]:
# BCE loss function
criterion = nn.BCELoss()
# feature matchin criterion
feature_matching_criterion = nn.MSELoss()

# fixed noise z for viusalization of the progress of the training
fixed_noise = torch.randn(batchSize, nz, 1, 1, device=device)

# setup optmization 
optimizerD = torch.optim.Adam(disc.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerG = torch.optim.Adam(gen.parameters(), lr=lr, betas=(beta1, 0.999))

In [347]:
# training loop 

# Lists to keep track of progress
img_list = []
G_losses = []
D_losses = []
iters = 0

print("Starting Training Loop...")

Starting Training Loop...


In [0]:
import numpy as np
dgz = 0 

bestGParams = None

In [0]:
# For each epoch
for epoch in range(num_epochs):
    # For each batch in the trainLoader
    for i, data in enumerate(trainLoader, 0):

        ############################
        # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################
        ## Train with all-real batch
        disc.zero_grad()
        # Format batch
        real_cpu = data[0].to(device)
        b_size = real_cpu.size(0)
        label_real = torch.full((b_size,), real_label, device=device)
        # Forward pass real batch through D
        output_real, feat_real1, feat_real2 = disc(real_cpu)
        output_real = output_real.view(-1)
        # Calculate loss on all-real batch
        errD_real = criterion(output_real, label_real)
        # Calculate gradients for D in backward pass
        
        D_x = output_real.mean().item()

        ## Train with all-fake batch
        # Generate batch of latent vectors
        noise = torch.randn(b_size, nz, 1, 1, device=device)

        # Generate fake image batch with G
        fake = gen(noise)
        label_fake = torch.full((b_size,), fake_label, device=device)
        # Classify all fake batch with D
        output_fake, feat_fake1, feat_fake2 = disc(fake.detach())
        output_fake = output_fake.view(-1)
        # Calculate D's loss on the all-fake batch
        errD_fake = criterion(output_fake, label_fake)
        # Calculate the gradients for this batch
        
        D_G_z1 = output_fake.mean().item()

        # feat matching errors
        feat_real1 = feat_real1.detach()  # so that PyTorch will treat them as volatile
        feat_real2 = feat_real2.detach()  # so that PyTorch will treat them as volatile
        fm_loss1 = feature_matching_criterion(feat_fake1, feat_real1)
        fm_loss2 = feature_matching_criterion(feat_fake1, feat_real1)

        # Add the gradients from the all-real and all-fake batches
        errD = 1 / 10 * (errD_real + errD_fake) + 6 / 10 * fm_loss1 + 3 / 10 * fm_loss2
        # errD = fm_loss

        # errD_real.backward()
        # errD_fake.backward()
        errD.backward()
        # Update D
        optimizerD.step()

        ############################
        # (2) Update G network: maximize log(D(G(z)))
        ###########################
        gen.zero_grad()
        label = torch.full((b_size,), real_label, device=device)  # fake labels are real for generator cost
        # Since we just updated D, perform another forward pass of all-fake batch through D
        output, _, _ = disc(fake)
        output = output.view(-1)
        # Calculate G's loss based on this output
        errG = criterion(output, label)
        # Calculate gradients for G
        errG.backward()
        D_G_z2 = output.mean().item()
        # Update G
        optimizerG.step()


        # Output training stats
        if i % 50 == 0:
            print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
                  % (epoch, num_epochs, i, len(trainLoader),
                     errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))

        # Save Losses for plotting later
        G_losses.append(errG.item())
        D_losses.append(errD.item())

        # Check how the generator is doing by saving G's output on fixed_noise
        if (iters % 500 == 0) or ((epoch == num_epochs-1) and (i == len(trainLoader)-1)):
            clear_output()
            with torch.no_grad():
                fake = gen(fixed_noise).detach().cpu()
            img_list.append(vutils.make_grid(fake, padding=2, normalize=True))

        iters += 1

[0/5][3550/6250]	Loss_D: 0.2804	Loss_G: 2.2494	D(x): 0.8286	D(G(z)): 0.8697 / 0.0900
[0/5][3600/6250]	Loss_D: 0.1406	Loss_G: 1.0802	D(x): 0.5669	D(G(z)): 0.4257 / 0.3225
[0/5][3650/6250]	Loss_D: 0.1355	Loss_G: 1.6686	D(x): 0.5467	D(G(z)): 0.3498 / 0.1666
[0/5][3700/6250]	Loss_D: 0.1799	Loss_G: 1.5189	D(x): 0.7248	D(G(z)): 0.6566 / 0.1966
[0/5][3750/6250]	Loss_D: 0.1667	Loss_G: 1.2044	D(x): 0.5136	D(G(z)): 0.5057 / 0.2831
[0/5][3800/6250]	Loss_D: 0.1407	Loss_G: 1.0321	D(x): 0.5211	D(G(z)): 0.4275 / 0.3336
[0/5][3850/6250]	Loss_D: 0.1607	Loss_G: 1.8322	D(x): 0.7377	D(G(z)): 0.5934 / 0.1392
[0/5][3900/6250]	Loss_D: 0.1477	Loss_G: 1.7246	D(x): 0.6535	D(G(z)): 0.5233 / 0.1542
[0/5][3950/6250]	Loss_D: 0.1333	Loss_G: 1.7599	D(x): 0.6520	D(G(z)): 0.4495 / 0.1512


In [0]:
len(G_losses)

In [0]:
plt.figure(figsize=(10,5))
plt.title("Generator and Discriminator Loss During Training")
plt.plot(G_losses,label="G")
plt.plot(D_losses,label="D")
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [0]:
trainIter = iter(trainLoader)

In [0]:
real_batch = next(trainIter)

noise = torch.randn(64, nz, 1, 1, device=device)
fake = gen(noise).detach().cpu()
fake = vutils.make_grid(fake, padding=2, normalize=True)

# Plot the real images
plt.figure(figsize=(15,15))
plt.subplot(1,2,1)
plt.axis("off")
plt.title("Real Images")
plt.imshow(np.transpose(vutils.make_grid(real_batch[0].to(device)[:64], padding=5, normalize=True).cpu(),(1,2,0)))

# Plot the fake images from the last epoch
plt.subplot(1,2,2)
plt.axis("off")
plt.title("Fake Images")
plt.imshow(np.transpose(fake,(1,2,0)))
plt.show()

In [0]:
print(optimizerD, optimizerG, trainLoader.batch_size, ngf, ndf, real_label)

In [0]:
genParams = gen.state_dict()
discParams = disc.state_dict()

In [0]:
def dumpAndDL(variable, fileName):
    with open(fileName, 'wb') as f:
        pickle.dump(variable,
                    f)
        files.download(fileName)
        
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [0]:
dumpAndDL(
    {'genParams': genParams,
     'discParams': discParams,
    },
    'bestParams.p'
)

In [0]:
dumpAndDL(
    {'G_losses': G_losses,
     'D_losses':D_losses,
     },
     'losses.p'
       )

In [0]:
n=50
for idx, chunk in enumerate(chunks(img_list, n)):
    fileName = 'img_list_{}.p'.format(idx)
    dumpAndDL(
        chunk,
        fileName
    )

In [0]:
# # load params to networks
# fileName = 'bestParams.p'
# netParams = None
# with open(fileName, 'rb') as f:
#      netParams = pickle.load(f)
# gen.load_state_dict(netParams['genParams'])
# disc.load_state_dict(netParams['discParams'])

In [0]:
# load params to networks
fileNames = ['img_list_0.p', 'img_list_1.p', 'img_list_2.p', 'img_list_3.p']

img_list = []
for fileName in fileNames:
    with open(fileName, 'rb') as f:
        img_list += pickle.load(f)['img_list']


In [0]:
#%%capture
fig = plt.figure(figsize=(10, 10))
plt.axis("off")
ims = [[plt.imshow(np.transpose(i,(1,2,0)), animated=True)] for i in img_list]
ani = animation.ArtistAnimation(fig, ims, interval=100, repeat_delay=1000, blit=True)

# HTML(ani.to_jshtml())

In [0]:
Writer = animation.writers['ffmpeg']
ani.save('im.mp4', writer=writer)